In [48]:
import tensorflow as tf

import numpy as np
import os
import time

In [49]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'http://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')

In [50]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [51]:
print('Length of text: {} characters.'.format(len(text)))

Length of text: 99993 characters.


In [52]:
print(text[:250])

That, poor contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, kno


In [53]:
vocab = sorted(set(text))
print('Number of unique characters: {}'.format(len(vocab)))

Number of unique characters: 62


In [54]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [55]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('   ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
   ...
}


In [56]:
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'That, poor co' ---- characters mapped to int ---- > [29 43 36 55  4  1 51 50 50 53  1 38 50]


In [57]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
h
a
t
,


In [58]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

"That, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their defe"
'ated queen,\nHer flesh broke me and puttance of expedition house,\nAnd in that same that ever I lament '
'this stomach,\nAnd he, nor Butly and my fury, knowing everything\nGrew daily ever, his great strength a'
"nd thought\nThe bright buds of mine own.\n\nBIONDELLO:\nMarry, that it may not pray their patience.'\n\nKIN"
'G LEAR:\nThe instant common maid, as we may less be\na brave gentleman and joiner: he that finds us wit'


In [59]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [60]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "That, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their def"
Target data:  "hat, poor contempt, or claim'd thou slept so faithful,\nI may contrive our father; and, in their defe"


In [61]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  exptected output: {}  ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('T')
  exptected output: 43  ('h')
Step    1
  input: 43 ('h')
  exptected output: 36  ('a')
Step    2
  input: 36 ('a')
  exptected output: 55  ('t')
Step    3
  input: 55 ('t')
  exptected output: 4  (',')
Step    4
  input: 4 (',')
  exptected output: 1  (' ')


In [62]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [63]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [64]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [65]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [66]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 62) # (batch_size, sequence_length, vocab_size)


In [67]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           15872     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 62)            63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


In [68]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [69]:
sampled_indices

array([36, 23,  4, 33, 28, 14, 27, 46,  8, 46, 43, 39,  1, 58, 44, 61, 54,
       31,  3,  8, 18, 17, 52, 38, 19, 56, 29, 34, 53, 34, 33, 12, 30, 58,
       59, 13, 28, 42, 28,  5, 52, 13, 19,  7, 44, 36,  4,  6, 53, 42, 56,
       45, 34, 33,  1,  4, 50, 23, 16, 61, 43, 57, 29, 35, 11, 30, 40, 21,
       38, 57, 46, 12, 24, 47,  5, 31, 48, 52, 17,  5, 29, 46, 48,  4, 40,
       47, 48, 48,  1, 19, 35, 52, 10, 31, 41, 59, 15, 55,  1,  2],
      dtype=int64)

In [70]:
print("Input: \n", repr(''.join(idx2char[input_example_batch[0]])))
print()
print("Next character predictions: \n", repr(''.join(idx2char[sampled_indices ])))

Input: 
 "ens and desired all\nFor what with mountain blasting phrase or wonder\nMakes lobes that stabb'd thy co"

Next character predictions: 
 "aN,XSERk;khd wizsV';IHqcJuTYrYXCUwxDSgS-qDJ:ia,.rgujYX ,oNGzhvTZBUeLcvkCOl-VmqH-Tkm,elmm JZqAVfxFt !"


In [71]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Scalar Loss: ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 62)  # (batch_size, sequence_length, vocab_size)
Scalar Loss:  4.1270747


In [72]:
model.compile(optimizer='adam', loss=loss)

In [73]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [74]:
EPOCHS=30

In [75]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
15/15 [==============================] - 8s 530ms/step - loss: 3.4697
Epoch 2/30
15/15 [==============================] - 8s 513ms/step - loss: 3.2028
Epoch 3/30
15/15 [==============================] - 8s 566ms/step - loss: 2.9923
Epoch 4/30
15/15 [==============================] - 8s 514ms/step - loss: 2.6418
Epoch 5/30
15/15 [==============================] - 8s 510ms/step - loss: 2.4580
Epoch 6/30
15/15 [==============================] - 8s 561ms/step - loss: 2.3577
Epoch 7/30
15/15 [==============================] - 8s 561ms/step - loss: 2.2804
Epoch 8/30
15/15 [==============================] - 8s 563ms/step - loss: 2.2137
Epoch 9/30
15/15 [==============================] - 8s 566ms/step - loss: 2.1562
Epoch 10/30
15/15 [==============================] - 8s 563ms/step - loss: 2.0970
Epoch 11/30
15/15 [==============================] - 8s 526ms/step - loss: 2.0416
Epoch 12/30
15/15 [==============================] - 9s 567ms/step - loss: 1.9898
Epoch 13/30
15/15 [======

In [76]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_30'

In [77]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [78]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_3 (Dense)              (1, None, 62)             63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


In [79]:
def generate_text(model, start_string):
    
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions /= temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [80]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: BLICDEUDu-SNLASMLGQIVIIUTC-UPSSBIX IPUSSANIA:
there's
mocheref shall be with lefare,
And you blooked of by their death-do.

MARK ANTONY:
Fo- to do.

CHENMINA:
Even ale my hood uppean beteem's ore: he's cause not and bewam
Ron thus he not old were shall adated.

MELUT:
Shall may we dray? To-pack to so hear my good in.

LADD HANK:
Now.

HINE PENNY
Moster'' timl father, now thet we would have come did the wirt, be manigld.

HOENE:
Thou mast fercomen:
Thin think of their vestence would not to a man learm
Tike for he's to our than morise fay not that with bearemoul
To am the rodam and diphink it our hose;
For not Hesh way not Tell and me then.

MARK ANTONY:
The pails 'at some swaiff and shorous in the leament loke,
The trust for in sufficcimed to thet he thine in thy magre
To must of me cape of you have sweet here,
An one insurberaveingers of Listabuse.
This will come thee athicr my forthian to till known,
Go love that this rust reloote to did wear in thy good loys,
Amacestod and tom

In [81]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)


In [82]:
optimizer = tf.keras.optimizers.Adam()

In [83]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [84]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))

    if (epoch+1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print('Time taken for a single epoch: {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.126844882965088
Epoch 1 Loss 3.2846
Time taken for a single epoch: 20.90825390815735 sec

Epoch 2 Batch 0 Loss 3.3290772438049316
Epoch 2 Loss 3.1706
Time taken for a single epoch: 19.96717882156372 sec

Epoch 3 Batch 0 Loss 4.0030364990234375
Epoch 3 Loss 3.0431
Time taken for a single epoch: 19.212836265563965 sec

Epoch 4 Batch 0 Loss 2.999776840209961
Epoch 4 Loss 2.6755
Time taken for a single epoch: 19.195477962493896 sec

Epoch 5 Batch 0 Loss 2.68159556388855
Epoch 5 Loss 2.4814
Time taken for a single epoch: 18.657591104507446 sec

Epoch 6 Batch 0 Loss 2.466998338699341
Epoch 6 Loss 2.3641
Time taken for a single epoch: 18.807147979736328 sec

Epoch 7 Batch 0 Loss 2.4043960571289062
Epoch 7 Loss 2.3069
Time taken for a single epoch: 19.169296741485596 sec

Epoch 8 Batch 0 Loss 2.320762872695923
Epoch 8 Loss 2.2867
Time taken for a single epoch: 19.17879056930542 sec

Epoch 9 Batch 0 Loss 2.2612082958221436
Epoch 9 Loss 2.2053
Time taken for a single epoch